In [ ]:
import pandas as pd
import openai
import time
from tqdm import tqdm

# Set up OpenAI client
client = openai.OpenAI(
    api_key=""
)

# Load CSV
df = pd.read_csv("data/Looking For Gamers (LFG) - 💬 Main Chats - general [178960147027197952].csv")
df["Game"] = ""

# Helper: Ask GPT in batch
def ask_batch_gpt(messages):
    prompt = "Please analyze the following Discord messages. For each one, extract games mentioned(only the full name. No explanation).\nIf there are none, write 'None'.\n\n"
    for i, msg in enumerate(messages, 1):
        prompt += f"Message {i}: \"{msg}\"\n"
    prompt += "\nFormat:\nMessage 1:\nGame: ...\nMessage 2:\n..."

    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
        )
        return response.choices[0].message.content
    except Exception as e:
        print("Error:", e)
        return ""

import re

def parse_response(response):
    results = {}
    chunks = response.split("Message ")
    for chunk in chunks:
        if not chunk.strip():
            continue
        lines = chunk.strip().splitlines()
        match = re.match(r"(\d+):?", lines[0].strip())
        if not match:
            continue
        idx = int(match.group(1))
        game = ""
        for line in lines[1:]:
            if line.lower().startswith("game"):
                game = line.split(":", 1)[-1].strip()
        results[idx] = game
    return results

In [15]:
# Batch process
batch_size = 10
row_cnt = 150000
subset = df.iloc[:row_cnt]

for start in tqdm(range(0, len(subset), batch_size)):
    batch = subset.iloc[start:start+batch_size]
    messages = batch["Content"].fillna("").astype(str).str.strip().tolist()
    response = ask_batch_gpt(messages)
    parsed = parse_response(response)

    for i, game in parsed.items():
        if start + i - 1 < len(subset):
            idx = subset.index[start + i - 1]
            df.at[idx, "Game"] = game

    time.sleep(1)  # Avoid rate limits



100%|██████████| 15000/15000 [12:32:22<00:00,  3.01s/it]  


In [16]:
# Save
df_out = subset[["AuthorID", "Author", "Content", "Game"]]
df_out.to_csv("discord_author_game.csv", index=False)
print("✅ Saved to discord_author_game.csv")

✅ Saved to discord_author_game.csv
